In [1]:
import os
os.chdir(os.path.join(os.getcwd(), '..'))

In [8]:
from utils.data_utils import load_recording
from utils.data_utils import plot_first_n, plot_spec
import torch
from tqdm import tqdm
import yaml
from models.bendr import EncoderConv, ContextNetwork

with open('configs/od_config_bendr.yaml', 'r') as file:
        cfg = yaml.safe_load(file)


encoder_sd = torch.load('/Users/ksc/Desktop/Нейротемка/checkpoints_70ep/encoder_2024-12-23T19:57:53.101820.pt', map_location=torch.device('cpu') )
context_network_sd = torch.load('/Users/ksc/Desktop/Нейротемка/checkpoints_70ep/context_network_2024-12-23T19:57:53.101820.pt', map_location=torch.device('cpu') )


encoder = EncoderConv(**cfg['encoder'])
context_network = ContextNetwork(**cfg['context_network'])

encoder.load_state_dict(encoder_sd)
context_network.load_state_dict(context_network_sd)
encoder.eval()
context_network.eval()

/var/folders/z2/jk8j2nn905v4_76cbzdgkrxm0000gn/T/ipykernel_1597/4240429068.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  encoder_sd = torch.load('/Users/ksc/Desktop/Н

ContextNetwork(
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-19): 20 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (target_proj): Linear(in_features=512, out_features=512, bias=True)
  (positional_emb): SinusoidalPositionalEncoding(
    (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=False)
  )
)

In [6]:
from dataset.labeled_dataset import EEGLabeledDataset
from torch.utils.data import DataLoader

dataset = EEGLabeledDataset(
    data_path='/Users/ksc/Downloads/neural_data/full_clean/h5',
    cache_processed_path='/Users/ksc/Downloads/neural_data/full_clean/cache',
    train_length=73728, dataset_mode="full",
    rebuild_cache=False, limit=None,
    target_config={
    "user_id": True,
    "activity": False, },
    clipped_threshold=100
)
loader = DataLoader(dataset, 4, shuffle=True, drop_last=False)
next(iter(loader))['data'].shape


torch.Size([4, 4, 73728])

In [9]:
# device = torch.device('mps')
# encoder = encoder.to(device)
# context_network = context_network.to(device)

batch = next(iter(loader))
# batch['data'] = batch['data'].to(device)
batch = encoder(batch)
batch = context_network(batch)
batch

{'data': tensor([[[-0.0343, -0.0338, -0.1790,  ..., -0.3259, -0.2878, -0.2375],
          [-0.1603, -0.0903, -0.0397,  ...,  0.1292,  0.2028,  0.2488],
          [-0.0218,  0.0086, -0.0207,  ..., -0.0352, -0.1657, -0.2809],
          [ 0.2166, -0.0094, -0.3853,  ..., -0.2639, -0.0165,  0.2232]],
 
         [[-0.2479, -0.3788, -0.4958,  ...,  0.3691,  0.5526,  0.5846],
          [ 0.0606, -0.2076, -0.4302,  ...,  0.0082,  0.3229,  0.6536],
          [-0.1981, -0.2386, -0.4338,  ...,  0.8104,  1.0669,  0.9036],
          [-0.3163, -0.6246, -0.8342,  ...,  0.0826,  0.3692,  0.4372]],
 
         [[ 0.0354,  0.5168,  0.8165,  ..., -0.3835, -0.7210, -1.2541],
          [ 0.8198,  0.8962,  0.6931,  ..., -0.4225, -0.6454, -0.8414],
          [ 1.0822,  1.1161,  1.0196,  ..., -0.0672,  0.1374,  0.1797],
          [ 1.0532,  1.0125,  1.0467,  ..., -0.8097, -0.7751, -0.4627]],
 
         [[ 3.6740,  3.4682,  3.1442,  ..., -0.6784, -0.6527, -0.7135],
          [ 3.5529,  3.2017,  2.7408,  ..., -0.